卷积神经网络（Convolutional Neural Network，CNN）是一种专门用来处理具有类似网格结构数据的神经网络。例如时间序列数据（时间轴上有规律地采样形成的一维网格）和图像数据（二维的像素网格）。因此它被广泛用于图像识别、语音识别等各种场合，在图像识别的比赛中，基于深度学习的方法几乎都以 CNN 为基础。

# 1. 整体结构

之前介绍的神经网络中，相邻层的所有神经元之间都有连接，这称为全连接（Fully Connected）层或 Affine 层。全连接层后面跟着激活函数 ReLU 层（或者Sigmoid 层）。这里堆叠了4 层“Affine-ReLU”组合，然后第 5 层是全连接层，最后由 Softmax 层输出最终分类结果。

![img](images/chapter13/fully_connected.png)

CNN 中新增了Convolution 层和Pooling 层。典型 CNN 的层连接顺序是“Convolution - ReLU -（Pooling）”（Pooling 层有时会被省略）。

![img](images/chapter13/CNN.png)

# 2. 卷积层

## 2.1 全连接层存在的问题

输入数据的形状被全连接层“忽视”了。例如，向全连接层输入图像数据时，需要将 3 维图像数据（宽、高和通道数）拉平为 1 维数据。实际上，前面使用的 MNIST 数据集的例子中，输入图像就是 1 通道、高 28 像素、长 28 像素的（1, 28, 28）形状，但却被排成 1 列，以 784 个数据的形式输入到全连接层中。

图像的 3 维形状中含有重要的空间信息。例如，空间上邻近的像素为相似的值、RBG的各个通道之间分别有密切的关联性、相距较远的像素之间没有什么关联等，3 维形状中可能隐藏有值得提取的本质模式。但是，因为全连接层会忽视形状，将全部的输入数据作为相同的神经元（同一维度的神经元）处理，所以无法利用与形状相关的信息。

而卷积层可以保持数据形状不变。当输入数据是图像时，卷积层会以 3 维数据的形式接收输入数据，并同样以 3 维数据的形式输出至下一层。因此，在 CNN 中，可以正确理解图像等具有形状的数据。

CNN 中将卷积层的输出数据称为特征图（feature map）。

## 2.2 卷积运算

对于输入数据，卷积运算以一定间隔滑动卷积核（或称为滤波器）的窗口并应用。将各个位置上卷积核的元素和输入的对应元素相乘，然后再求和（有时将这个计算称为乘积累加运算）。然后，将这个结果保存到输出的对应位置。将这个过程在所有位置都进行一遍，就可以得到卷积运算的输出。如果将一个二维数据定义为 $I$，二维卷积核定义为 $K$，则

$$ S(i,j) = (I *K)(i,j) = \sum_m \sum_n I(i+m,j+n)K(m,n)$$

假设用（height, width）表示数据和卷积核的形状，则下图中输入数据尺寸是(5, 5)，输出数据尺寸是(3, 3)，卷积核尺寸是(3, 3)，其数值为

$$ \Bigg(
   \begin{matrix}
   0 & 1 & 2 \\
   2 & 2 & 0 \\
   0 & 1 & 2
  \end{matrix} \Bigg)
$$

![img](images/chapter13/numerical_no_padding_no_strides.gif)

令 $i$ 为输入数据尺寸，$k$ 为卷积核尺寸，$o$ 为输出数据尺寸，则有关系：

$$ o = (i - k) + 1 $$

当卷积层数增加时，网络的空间维度最终会缩减到1×1，这种情况下增加的层就不可能进行有意义的卷积了。

在全连接的神经网络中，除了权重参数，还存在偏置。CNN 中，卷积核的参数就对应之前的权重。并且，CNN 中也存在偏置，偏置被加到应用了卷积核的所有元素上。

## 2.3 填充（padding）

在进行卷积层的处理之前，有时要向输入数据的周围填入固定的数据（通常为 0），这称为**填充**（padding），是卷积运算中经常会用到的处理。在下图中，对大小为 (4, 4) 的输入数据应用了幅度为 1 的填充。通过填充，大小为 (4, 4) 的输入数据变成了 (6, 6) 的形状。然后应用大小为 (3, 3) 的卷积核，生成了大小为 (4, 4) 的输出数据。

![img](images/chapter13/padding.png)

令 $p$ 为填充幅度，则有关系

$$ o = (i - k) + 2p + 1 $$

下图中的示例中，$i=5, k =4, p =2$，因此 $o=6$

![img](images/chapter13/arbitrary_padding_no_strides.gif)

### Same Padding

一类特殊的填充作为称之为 same padding，即使得输出数据尺寸等于输入数据尺寸。令卷积核尺寸 $k$ 为奇数（$k = 2n+1$），填充幅度 $p = \lfloor \frac{k}{2} \rfloor= n$，则有

$$ o = (i - 2n - 1)+2n+1 = i $$

下图中的示例中，$i=o=5, k = 3, p=1$

![img](images/chapter13/same_padding_no_strides.gif)

在这种情况下，只要硬件支持，网络就能包含任意多的卷积层，这是因为卷积运算不改变下一层的结构。然而，输入数据中靠近边界的部分相比于中间部分对于输出数据的影响更小。这可能会导致边界像素存在一定程度的欠表示。

### Full Padding

另一类特殊的填充作为称之为 full padding，即使卷积核与数据刚相交时开始做卷积，因此填充幅度 $p = k - 1$，

$$o = (i - k)+2(k-1)+1 = i+k-1$$

下图中的示例中，$i=5, k = 3, p=2$，因此 $o=6$
![img](images/chapter13/full_padding_no_strides.gif)

它进行了足够多的零填充，使得每个像素在每个方向上恰好被访问了 $k$ 次。

## 2.4 步幅

应用卷积核的位置间隔称为**步幅**（stride）。之前的例子中步幅都是 1，如果将步幅设为 2，则如下图所示，应用卷积核的窗口的间隔变为 2 个元素。

![img](images/chapter13/stride.png)

令 $i$ 为输入数据尺寸，$p$ 为填充幅度，$k$ 为卷积核尺寸，$s$ 为步幅大小，$o$ 为输出数据尺寸，则有：

$$ o = \lfloor \frac{i + 2p -k}{s} \rfloor + 1 $$

下图中的示例中，$i=5, k=3, s=2, p=1$，因此 $o=3$

![img](images/chapter13/padding_strides.gif)

## 2.5 三维数据的卷积运算

图像是三维数据，将其表示为多维数组时顺序为(channel, height, width)，例如通道数为 C、高度为 H、长度为 W 的数据的形状可以写成（C, H, W）。在进行卷积运算时，除了高、宽方向之外还需要处理通道方向。通道方向上有多个特征图时，会按通道进行输入数据和卷积核的卷积运算，并将结果相加，从而得到输出。需要注意的是，输入数据和卷积核的通道数要为相同的值。

![img](images/chapter13/3D_conv_0.png)

这里以 3 通道的数据为例，输入数据尺寸是(3, 5, 5)，卷积核尺寸为(3, 3, 3)，填充幅度为0，步幅为1，则输出数据尺寸为(2, 2)。

![img](images/chapter13/3D_conv_1.png)

分步计算顺序如下所示：

![img](images/chapter13/3D_conv_2.png)

如果要在通道方向上也拥有多个卷积运算的输出，就需要用到多个卷积核。如下图所示，通过应用 FN 个滤波器，输出特征图也生成了 FN 个。如果将这 FN 个特征图汇集在一起，就得到了形状为(FN, OH,OW) 的数据体。

![img](images/chapter13/multi_channel_conv.png)

进一步完善包含偏置加法运算的卷积运算，如下图所示。卷积核输出结果的形状是(FN, OH,OW) ，偏置的形状是(FN, 1, 1)，这两个数据相加时，得益于 NumPy 的广播功能，按通道加上相同的偏置值。

![img](images/chapter13/multi_channel_conv_with_bias.png)

## 2.6 批处理

之前的全连接神经网络的实现对应了批处理，通过批处理能够实现高效化的运算和学习时对应 mini-batch 的 SDG 算法。卷积运算也同样支持批处理。为此，需要将在各层间传递的数据保存为四维数据，按(batch_num, channel, height, width)的顺序保存数据。

![img](images/chapter13/batch_conv.png)

## 2.7 卷积层的动机

卷积运算通过三个重要思想来帮助改进深度学习系统：**稀疏连接**（sparse connectivity）、**参数共享**（parameter sharing）、**等变表示**（equivariant representations）。

### 2.7.1 稀疏连接

传统的神经网络的全连接层使用矩阵乘法来建立输入与输出的连接关系。其中，参数矩阵中每一个单独的参数都描述了一个输入单元与一个输出单元间的交互。这意味着每一个输出单元与每一个输入单元都产生交互。然而，卷积网络具有稀疏连接（也叫作稀疏权重（sparse weights））的特征。这是通过使核的大小远小于输入数据大小来达到的。

例如，有 $m$ 个输入和 $n$ 个输出，那么全连接层的矩阵乘法需要 $m×n$ 个参数并且相应算法的时间复杂度为 $O(m×n)$。如果我们限制每一个输出拥有的连接数为 $k$，
那么稀疏的连接方法只需要 $k×n$ 个参数以及 $O(k×n)$ 的运行时间。在很多实际应用中，只需保持 $k$ 比 $m$ 小几个数量级，就能在机器学习的任务中取得好的表现。

![img](images/chapter13/sparse_connectivity.png)

在深度卷积网络中，处在网络深层的单元可能与绝大部分输入是间接交互的，这允许网络可以通过只描述稀疏连接的基石来高效地描述多个变量的复杂交互。下图中，灰色区域凸显了 $g3$ 神经元以及它的**感受野**（receptive field）。

![img](images/chapter13/receptive_field.png)

### 2.7.2 参数共享

参数共享是指在一个模型的多个函数中使用相同的参数。

在传统的神经网络中，当计算一层的输出时，权重矩阵的每一个元素只使用一次，当它乘以输入的一个元素后就再也不会用到了。

在卷积神经网络中，核的每一个元素都作用在输入的每一位置上。卷积运算中的参数共享保证了我们只需要学习一个参数集合，而不是对于每一位置都需要学习一个单独的参数集合。这虽然没有改变前向传播的运行时间（仍然是 $O(k×n)$），但它显著地把模型的存储需求降低至 $k$ 个参数，并且 $k$通常要比 $m$ 小很多个数量级。因此，卷积在存储需求和统计效率方面极大地优于稠密矩阵的乘法运算。

下图中的上半部分，单独的黑色箭头表示在全连接模型中对权重矩阵的中间元素的使用。这个模型没有使用参数共享，所以参数只使用了一次。而在卷积模型中因为参数共享，这个单独的参数被用于所有的输入位置。

![img](images/chapter13/parameter_sharing.png)

### 2.7.3 等变表示

参数共享的特殊形式使得神经网络层具有对**平移等变**（equivariance）的性质。如果一个函数满足输入改变，输出也以同样的方式改变这一性质，我们就说它是等变（equivariant）的。特别的是，如果函数 $f(x)$ 与 $g(x)$ 满足 $f(g(x))＝g(f(x))$，我们就说 $f(x)$ 对于变换 $g$ 具有等变性。对于卷积来说，如果令 $g$ 是输入的任意平移函数，那么卷积函数对于 $g$ 具有等变性。

例如，令 $I$ 表示图像在整数坐标上的亮度函数，$g$ 表示图像函数的变换函数使得 $I'＝g(I)$，其中图像函数 $I′$ 满足 $I′(x,y )＝I(x-1，y)$。这个函数把 $I$ 中的每个像素向右移动一个单位。如果我们先对 $I$ 进行这种变换然后进行卷积操作所得到的结果，与先对 $I$ 进行卷积然后再对输出使用平移函数 $g$ 得到的结果是一样的。

# 3. 池化层

## 池化层的特征

- 池化只是从目标区域中取最大值（或者平均值），所以不存在要学习的参数。


- 经过池化运算，输入数据和输出数据的通道数不会发生变化。


- 对微小的位置变化具有鲁棒性
      输入数据发生微小偏差时，池化仍会返回相同的结果。